##### Installation Required for pip, tensorflow

In [1]:
# Requires the latest pip
%pip install --upgrade pip --user

# Current stable release for CPU and GPU
%pip install tensorflow --user


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# TensorFlow 2 quickstart for beginners

這個簡短的介紹使用[Keras](https://www.tensorflow.org/guide/keras/overview)來實現以下功能：

1. 載入一個預先建立的數據集。
2. 構建一個神經網絡機器學習模型來對圖片進行分類。
3. 訓練這個神經網絡模型。
4. 評估模型的準確性。

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.13.0


If you are following along in your own development environment, rather than [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), see the [install guide](https://www.tensorflow.org/install) for setting up TensorFlow for development.

Note: Make sure you have upgraded to the latest `pip` to install the TensorFlow 2 package if you are using your own development environment. See the [install guide](https://www.tensorflow.org/install) for details.

## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`. This also converts the sample data from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build a machine learning model

Build a `tf.keras.Sequential` model:

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.16082239,  0.8892685 ,  0.30455583, -0.5361299 ,  0.8485933 ,
         0.05062216, -0.24663743,  0.58794236, -0.17622665, -0.6984148 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.09136591, 0.18929726, 0.10548891, 0.04550946, 0.18175204,
        0.08183231, 0.06078923, 0.14004901, 0.06522373, 0.03869209]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.503083

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2965 - accuracy: 0.9138
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1446 - accuracy: 0.9565
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1066 - accuracy: 0.9677
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0880 - accuracy: 0.9730
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0739 - accuracy: 0.9769


The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0725 - accuracy: 0.9775 - 334ms/epoch - 1ms/step


[0.072470523416996, 0.9775000214576721]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.56724607e-08, 7.40887103e-08, 5.65051050e-06, 1.07212709e-05,
        4.60259122e-11, 6.82544936e-08, 2.11188909e-13, 9.99981284e-01,
        3.38224879e-07, 1.80292773e-06],
       [3.29229088e-08, 1.68277384e-05, 9.99962091e-01, 4.22182165e-06,
        2.36436965e-16, 1.64210196e-05, 1.57122003e-07, 8.54547769e-13,
        3.33455290e-07, 3.83786461e-14],
       [2.27649991e-07, 9.98251021e-01, 5.67372481e-04, 1.60389993e-06,
        5.08309386e-05, 8.27862823e-06, 9.01208332e-05, 4.47708968e-04,
        5.77296596e-04, 5.45365720e-06],
       [9.99836564e-01, 1.04665186e-10, 3.41090708e-05, 4.56106228e-08,
        4.73024869e-07, 4.61766649e-06, 2.27378423e-05, 9.97211100e-05,
        1.67594982e-09, 1.70570547e-06],
       [1.30745520e-05, 1.95094496e-09, 1.95874265e-04, 1.61441392e-06,
        9.91898775e-01, 2.21241794e-06, 9.63726507e-07, 1.25521867e-04,
        4.44206262e-06, 7.75737641e-03]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).
